# Coffee Shop near British Museum
## Download Data

In [1]:
# import necessary libraries
import pandas as pd
import numpy as np
import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.18.1                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


### Define Foursquare Credentials and Version

In [2]:
CLIENT_ID = 'HCIWEYMLE0SJAI3ESV4AFX5PNQVBSLP5HQ1YU4GISAHHRIFV' # your Foursquare ID
CLIENT_SECRET = 'P4KVBEVJDIVREULUPIZHUL124JX353PUIP5KWJOGX1PLDB5B' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 50

### Search for a specific venue category

#### Find British Museum's latitude and longitude

In [3]:
address = 'Great Russell St, Bloomsbury, London WC1B 3DG, UK'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


51.5184748 -0.1281749


#### Search for coffee that is within 500 metres from the British Museum and define the corresponding URL

In [4]:
search_query = 'coffee'
radius = 500
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=HCIWEYMLE0SJAI3ESV4AFX5PNQVBSLP5HQ1YU4GISAHHRIFV&client_secret=P4KVBEVJDIVREULUPIZHUL124JX353PUIP5KWJOGX1PLDB5B&ll=51.5184748,-0.1281749&v=20180604&query=coffee&radius=500&limit=50'

#### Send the GET Request and examine the results

In [5]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c1b9acedb04f53ab43dc913'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d1e0931735',
      'name': 'Coffee Shop',
      'pluralName': 'Coffee Shops',
      'primary': True,
      'shortName': 'Coffee Shop'}],
    'hasPerk': False,
    'id': '4ac518edf964a520c1ac20e3',
    'location': {'address': '27 Monmouth St',
     'cc': 'GB',
     'city': 'Holborn and Covent Garden',
     'country': 'United Kingdom',
     'distance': 472,
     'formattedAddress': ['27 Monmouth St',
      'Holborn and Covent Garden',
      'Greater London',
      'WC2H 9EU',
      'United Kingdom'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 51.514314,
       'lng': -0.126824}],
     'lat': 51.514314,
     'lng': -0.126824,
     'postalCode': 'WC2H 9EU',
     'state': 'Greater London'},
    'name': 'Monmouth Coffee Company',
    'referra

#### get relevant part of JSON and transform it into a pandas dataframe

In [6]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# transform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'shortName': 'Coffee Shop', 'pluralName': 'C...",False,4ac518edf964a520c1ac20e3,27 Monmouth St,GB,Holborn and Covent Garden,United Kingdom,NaN,472,"[27 Monmouth St, Holborn and Covent Garden, Gr...","[{'label': 'display', 'lat': 51.514314, 'lng':...",51.514314,-0.126824,NaN,WC2H 9EU,Greater London,Monmouth Coffee Company,v-1545312974,NaN
1,"[{'shortName': 'Coffee Shop', 'pluralName': 'C...",False,4bbb2ff71261d13a1058eb98,26 Rathbone Pl,GB,Fitzrovia,United Kingdom,Percy St,424,"[26 Rathbone Pl (Percy St), Fitzrovia, Greater...","[{'label': 'display', 'lat': 51.51779541476858...",51.517795,-0.134211,Fitzrovia,W1T 1JD,Greater London,TAP Coffee No. 26,v-1545312974,NaN
2,"[{'shortName': 'Coffee Shop', 'pluralName': 'C...",False,4d7b8859645ea35daeb133f8,39 Shelton St,GB,Covent Garden,United Kingdom,NaN,544,"[39 Shelton St, Covent Garden, Greater London,...","[{'label': 'display', 'lat': 51.51407186153086...",51.514072,-0.124764,NaN,WC2H 9HJ,Greater London,Costa Coffee,v-1545312974,NaN
3,"[{'shortName': 'Coffee Shop', 'pluralName': 'C...",False,5221d93b11d2750f339082be,9 Windmill St,GB,Fitzrovia,United Kingdom,NaN,405,"[9 Windmill St, Fitzrovia, Greater London, W1T...","[{'label': 'display', 'lat': 51.51880890491309...",51.518809,-0.134006,NaN,W1T 2JF,Greater London,Sharps Coffee Bar,v-1545312974,NaN
4,"[{'shortName': 'Café', 'pluralName': 'Cafés', ...",False,4cc2c86b8062468853762b2f,Dean St,GB,Soho,United Kingdom,NaN,522,"[Dean St, Soho, Greater London, United Kingdom]","[{'label': 'display', 'lat': 51.51527327636101...",51.515273,-0.133681,NaN,NaN,Greater London,Soho Coffee Club,v-1545312974,NaN


#### Define information of interest and filter dataframe

In [7]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state
0,Monmouth Coffee Company,Coffee Shop,27 Monmouth St,GB,Holborn and Covent Garden,United Kingdom,NaN,472,"[27 Monmouth St, Holborn and Covent Garden, Gr...","[{'label': 'display', 'lat': 51.514314, 'lng':...",51.514314,-0.126824,NaN,WC2H 9EU,Greater London
1,TAP Coffee No. 26,Coffee Shop,26 Rathbone Pl,GB,Fitzrovia,United Kingdom,Percy St,424,"[26 Rathbone Pl (Percy St), Fitzrovia, Greater...","[{'label': 'display', 'lat': 51.51779541476858...",51.517795,-0.134211,Fitzrovia,W1T 1JD,Greater London
2,Costa Coffee,Coffee Shop,39 Shelton St,GB,Covent Garden,United Kingdom,NaN,544,"[39 Shelton St, Covent Garden, Greater London,...","[{'label': 'display', 'lat': 51.51407186153086...",51.514072,-0.124764,NaN,WC2H 9HJ,Greater London
3,Sharps Coffee Bar,Coffee Shop,9 Windmill St,GB,Fitzrovia,United Kingdom,NaN,405,"[9 Windmill St, Fitzrovia, Greater London, W1T...","[{'label': 'display', 'lat': 51.51880890491309...",51.518809,-0.134006,NaN,W1T 2JF,Greater London
4,Soho Coffee Club,Café,Dean St,GB,Soho,United Kingdom,NaN,522,"[Dean St, Soho, Greater London, United Kingdom]","[{'label': 'display', 'lat': 51.51527327636101...",51.515273,-0.133681,NaN,NaN,Greater London
5,Notes Coffee Barrows,Coffee Shop,60 St Giles High St,GB,Greater London,United Kingdom,NaN,329,"[60 St Giles High St, Greater London, WC2H 8LG...","[{'label': 'display', 'lat': 51.51553615403286...",51.515536,-0.128688,NaN,WC2H 8LG,Greater London
6,Costa Coffee,Coffee Shop,104 New Oxford St,GB,London,United Kingdom,NaN,208,"[104 New Oxford St, London, Greater London, WC...","[{'label': 'display', 'lat': 51.51663447291778...",51.516634,-0.128697,NaN,WC1A 1HB,Greater London
7,Better Coffee Corner,Coffee Shop,"Oasis Sports Centre, 32 Endell St",GB,London,United Kingdom,NaN,345,"[Oasis Sports Centre, 32 Endell St, London, Gr...","[{'label': 'display', 'lat': 51.515705, 'lng':...",51.515705,-0.125915,NaN,WC2H 9AG,Greater London
8,Costa Coffee,Coffee Shop,82 Gower St,GB,Camden Town,United Kingdom,in Waterstones,507,"[82 Gower St (in Waterstones), Camden Town, Gr...","[{'label': 'display', 'lat': 51.52223061484879...",51.522231,-0.132335,NaN,WC1E 6EQ,Greater London
9,Dillon's Coffee,Coffee Shop,82 Gower Street,GB,London,United Kingdom,NaN,538,"[82 Gower Street, London, Greater London, Unit...","[{'label': 'display', 'lat': 51.52250102265136...",51.522501,-0.132479,NaN,NaN,Greater London


#### DataFrame Pre-processing

In [8]:
# remove rows with distance > 500 m
dataframe_filtered = dataframe_filtered[dataframe_filtered[('distance')]<=500]
dataframe_filtered.reset_index(drop=True,inplace=True)

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state
0,Monmouth Coffee Company,Coffee Shop,27 Monmouth St,GB,Holborn and Covent Garden,United Kingdom,NaN,472,"[27 Monmouth St, Holborn and Covent Garden, Gr...","[{'label': 'display', 'lat': 51.514314, 'lng':...",51.514314,-0.126824,NaN,WC2H 9EU,Greater London
1,TAP Coffee No. 26,Coffee Shop,26 Rathbone Pl,GB,Fitzrovia,United Kingdom,Percy St,424,"[26 Rathbone Pl (Percy St), Fitzrovia, Greater...","[{'label': 'display', 'lat': 51.51779541476858...",51.517795,-0.134211,Fitzrovia,W1T 1JD,Greater London
2,Sharps Coffee Bar,Coffee Shop,9 Windmill St,GB,Fitzrovia,United Kingdom,NaN,405,"[9 Windmill St, Fitzrovia, Greater London, W1T...","[{'label': 'display', 'lat': 51.51880890491309...",51.518809,-0.134006,NaN,W1T 2JF,Greater London
3,Notes Coffee Barrows,Coffee Shop,60 St Giles High St,GB,Greater London,United Kingdom,NaN,329,"[60 St Giles High St, Greater London, WC2H 8LG...","[{'label': 'display', 'lat': 51.51553615403286...",51.515536,-0.128688,NaN,WC2H 8LG,Greater London
4,Costa Coffee,Coffee Shop,104 New Oxford St,GB,London,United Kingdom,NaN,208,"[104 New Oxford St, London, Greater London, WC...","[{'label': 'display', 'lat': 51.51663447291778...",51.516634,-0.128697,NaN,WC1A 1HB,Greater London
5,Better Coffee Corner,Coffee Shop,"Oasis Sports Centre, 32 Endell St",GB,London,United Kingdom,NaN,345,"[Oasis Sports Centre, 32 Endell St, London, Gr...","[{'label': 'display', 'lat': 51.515705, 'lng':...",51.515705,-0.125915,NaN,WC2H 9AG,Greater London
6,Coffee Lounge 2F,Coffee Shop,NaN,GB,London,United Kingdom,2F,130,"[2F, London, Greater London, W C1B, United Kin...","[{'label': 'display', 'lat': 51.518982, 'lng':...",51.518982,-0.126474,NaN,W C1B,Greater London
7,Costa Coffee,Coffee Shop,34 Rathbone Place,GB,London,United Kingdom,NaN,430,"[34 Rathbone Place, London, Greater London, W1...","[{'label': 'display', 'lat': 51.51759, 'lng': ...",51.517590,-0.134224,West End,W1T 1JN,Greater London
8,Black Sheep Coffee,Coffee Shop,5-7 Goodge St,GB,NaN,United Kingdom,NaN,446,"[5-7 Goodge St, W1T 2PD, United Kingdom]","[{'label': 'display', 'lat': 51.52003491351633...",51.520035,-0.134121,NaN,W1T 2PD,NaN
9,The Coffee Tree,Coffee Shop,222 Shaftesbury Ave,GB,London,United Kingdom,NaN,245,"[222 Shaftesbury Ave, London, Greater London, ...","[{'label': 'display', 'lat': 51.51656150600664...",51.516562,-0.126427,NaN,WC2H 8EB,Greater London


Seems like there are quite many coffee shops in radius 500m. Try to reduce to 400 m distance. 

In [9]:
# remove rows with distance > 400 m
dataframe_filtered = dataframe_filtered[dataframe_filtered[('distance')]<=400]
dataframe_filtered.reset_index(drop=True,inplace=True)

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state
0,Notes Coffee Barrows,Coffee Shop,60 St Giles High St,GB,Greater London,United Kingdom,NaN,329,"[60 St Giles High St, Greater London, WC2H 8LG...","[{'label': 'display', 'lat': 51.51553615403286...",51.515536,-0.128688,NaN,WC2H 8LG,Greater London
1,Costa Coffee,Coffee Shop,104 New Oxford St,GB,London,United Kingdom,NaN,208,"[104 New Oxford St, London, Greater London, WC...","[{'label': 'display', 'lat': 51.51663447291778...",51.516634,-0.128697,NaN,WC1A 1HB,Greater London
2,Better Coffee Corner,Coffee Shop,"Oasis Sports Centre, 32 Endell St",GB,London,United Kingdom,NaN,345,"[Oasis Sports Centre, 32 Endell St, London, Gr...","[{'label': 'display', 'lat': 51.515705, 'lng':...",51.515705,-0.125915,NaN,WC2H 9AG,Greater London
3,Coffee Lounge 2F,Coffee Shop,NaN,GB,London,United Kingdom,2F,130,"[2F, London, Greater London, W C1B, United Kin...","[{'label': 'display', 'lat': 51.518982, 'lng':...",51.518982,-0.126474,NaN,W C1B,Greater London
4,The Coffee Tree,Coffee Shop,222 Shaftesbury Ave,GB,London,United Kingdom,NaN,245,"[222 Shaftesbury Ave, London, Greater London, ...","[{'label': 'display', 'lat': 51.51656150600664...",51.516562,-0.126427,NaN,WC2H 8EB,Greater London
5,That Coffee Shop,Coffee Shop,Shaftesbury Avenue,GB,London,United Kingdom,NaN,236,"[Shaftesbury Avenue, London, Greater London, W...","[{'label': 'display', 'lat': 51.51692, 'lng': ...",51.516920,-0.125852,NaN,WC2H 8EG,Greater London
6,Insomnia Coffee Company,Coffee Shop,NaN,GB,London,United Kingdom,NaN,328,"[London, Greater London, W1D 1AW, United Kingdom]","[{'label': 'display', 'lat': 51.51644591899548...",51.516446,-0.131624,NaN,W1D 1AW,Greater London
7,Black Sheep Coffee,Coffee Shop,NaN,GB,London,United Kingdom,Earnshaw street,263,"[Earnshaw street, London, Greater London, W C2...","[{'label': 'display', 'lat': 51.516122, 'lng':...",51.516122,-0.128637,NaN,W C2H,Greater London
8,Starbucks,Coffee Shop,112-116 New Oxford St,GB,Bloomsbury,United Kingdom,NaN,219,"[112-116 New Oxford St, Bloomsbury, Greater Lo...","[{'label': 'display', 'lat': 51.516645, 'lng':...",51.516645,-0.129358,NaN,WC1A 1HH,Greater London
9,Coffee & Gift,Coffee Shop,NaN,GB,NaN,United Kingdom,NaN,213,[United Kingdom],"[{'label': 'display', 'lat': 51.517033, 'lng':...",51.517033,-0.126135,NaN,NaN,NaN


Seems like now the amount of coffee shop is suitable for me.     
Let me check the data columns

In [10]:
dataframe_filtered.columns

Index(['name', 'categories', 'address', 'cc', 'city', 'country', 'crossStreet',
       'distance', 'formattedAddress', 'labeledLatLngs', 'lat', 'lng',
       'neighborhood', 'postalCode', 'state'],
      dtype='object')

There are too many unnecessary columns in the dataframe. Only extract the information I need, including coffee shope name, latitude, longitude, and distance to British Museum.

In [11]:
coffee_shop = dataframe_filtered[['name','lat','lng','distance']]
coffee_shop

,name,lat,lng,distance
0,Notes Coffee Barrows,51.515536,-0.128688,329
1,Costa Coffee,51.516634,-0.128697,208
2,Better Coffee Corner,51.515705,-0.125915,345
3,Coffee Lounge 2F,51.518982,-0.126474,130
4,The Coffee Tree,51.516562,-0.126427,245
5,That Coffee Shop,51.516920,-0.125852,236
6,Insomnia Coffee Company,51.516446,-0.131624,328
7,Black Sheep Coffee,51.516122,-0.128637,263
8,Starbucks,51.516645,-0.129358,219
9,Coffee & Gift,51.517033,-0.126135,213


## Explore coffee shop in the map

In [12]:
venues_map = folium.Map(location=[51.5184748, -0.1281749], zoom_start=15) # generate map centred around the British Museum

# add a red circle marker to represent the Museum
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='British Museum',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

## Clustser the shop

In [13]:
# set number of clusters
kclusters = 3

coffee_shop_clustering = coffee_shop.drop('name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(coffee_shop_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 2, 0, 0, 1, 0, 0, 0], dtype=int32)

In [14]:
Cluster_Labels = pd.Series(kmeans.labels_)
coffee_shop = coffee_shop.assign(Cluster_Labels=Cluster_Labels.values)
coffee_shop.head()

,name,lat,lng,distance,Cluster_Labels
0,Notes Coffee Barrows,51.515536,-0.128688,329,1
1,Costa Coffee,51.516634,-0.128697,208,0
2,Better Coffee Corner,51.515705,-0.125915,345,1
3,Coffee Lounge 2F,51.518982,-0.126474,130,2
4,The Coffee Tree,51.516562,-0.126427,245,0


## Visualize the resulting clusters

In [15]:
# create map
map_clusters = folium.Map(location=[51.5184748, -0.1281749], zoom_start=15)

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=15,
    color='red',
    popup='British Museum',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(map_clusters)



# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(coffee_shop['lat'], coffee_shop['lng'], 
                                  coffee_shop['name'], 
                                  coffee_shop['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=7,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=1).add_to(map_clusters)
       
map_clusters

## Examine Clusters

In [16]:
Cluster0 = coffee_shop.loc[coffee_shop['Cluster_Labels'] == 0, coffee_shop.columns[[0,1,2,3] + list(range(5, coffee_shop.shape[1]))]]
Cluster0

,name,lat,lng,distance
1,Costa Coffee,51.516634,-0.128697,208
4,The Coffee Tree,51.516562,-0.126427,245
5,That Coffee Shop,51.516920,-0.125852,236
7,Black Sheep Coffee,51.516122,-0.128637,263
8,Starbucks,51.516645,-0.129358,219
9,Coffee & Gift,51.517033,-0.126135,213
10,Costa Coffee,51.516976,-0.126072,221


In [17]:
Cluster1 = coffee_shop.loc[coffee_shop['Cluster_Labels'] == 1, coffee_shop.columns[[0,1,2,3] + list(range(5, coffee_shop.shape[1]))]]
Cluster1

,name,lat,lng,distance
0,Notes Coffee Barrows,51.515536,-0.128688,329
2,Better Coffee Corner,51.515705,-0.125915,345
6,Insomnia Coffee Company,51.516446,-0.131624,328
12,Express Coffee Co,51.515956,-0.124060,399
14,Starbucks,51.515233,-0.130191,386


In [18]:
Cluster2 = coffee_shop.loc[coffee_shop['Cluster_Labels'] == 2, coffee_shop.columns[[0,1,2,3] + list(range(5, coffee_shop.shape[1]))]]
Cluster2

,name,lat,lng,distance
3,Coffee Lounge 2F,51.518982,-0.126474,130
11,Franks Coffee Shop,51.518022,-0.125672,180
13,Starbucks,51.518284,-0.125823,164
